# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Compare the results with normal holdout validation
- Apply 5-fold cross validation for regression

## Let's get started

This time, let's only include the variables that were previously selected using recursive feature elimination. We included the code to preprocess below.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import load_boston

boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
b = boston_features["B"]
logdis = np.log(boston_features["DIS"])
loglstat = np.log(boston_features["LSTAT"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))

In [2]:
X = boston_features[['CHAS', 'RM', 'DIS', 'B', 'LSTAT']]
y = pd.DataFrame(boston.target,columns = ['target'])

## Train test split

Perform a train-test-split with a test set of 0.20.

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [5]:
print(len(x_train), len(x_test), len(y_train), len(y_test))

404 102 404 102


Fit the model and apply the model to the make test set predictions

In [6]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()

In [7]:
result = linreg.fit(x_train, y_train)

In [8]:
y_hat_train = result.predict(x_train)
y_hat_test = result.predict(x_test)

Calculate the residuals and the mean squared error

In [9]:
from sklearn.metrics import mean_squared_error as mse

In [10]:
train_MSE = mse(y_train, y_hat_train)
test_MSE = mse(y_test, y_hat_test)

In [11]:
train_MSE

22.652349660324546

In [12]:
test_MSE

18.384562770971797

## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function k-folds that splits a dataset into k evenly sized pieces.
If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [33]:
def kfolds(data, k):
    # Force data as pandas dataframe
    # add 1 to fold size to account for leftovers
    data = pd.DataFrame(data)
    fold_size = len(data)//k
    remainder = len(data)%k
    folds = []
    start = 0
    for i in range(0,len(data), fold_size):
        if len(data) - i < fold_size:
            count = 0
            for j in range(start,len(data)):
                folds[count].append(data.iloc[start+count,:])
                count += 1
        else:
            folds.append(data[start:start + fold_size])
            start += fold_size
            
    return folds

### Apply it to the Boston Housing Data

In [34]:
# Make sure to concatenate the data again
len(bos_data)

506

In [35]:
bos_data = pd.concat([X.reset_index(drop=True), y], axis=1)

In [36]:
folds = kfolds(bos_data, 5)

### Perform a linear regression for each fold, and calculate the training and test error

Perform linear regression on each and calculate the training and test error.

In [136]:
test_errs = []
train_errs = []
k=5

for n in range(k):
    # Split in train and test for the fold
    train = pd.concat([folds[:n], folds[n:]], axis = 0)
    test = None
    # Fit a linear regression model
    
    #Evaluate Train and Test Errors

# print(train_errs)
# print(test_errs)

TypeError: cannot concatenate object of type "<class 'list'>"; only pd.Series, pd.DataFrame, and pd.Panel (deprecated) objs are valid

In [41]:
pd.concat([folds[0]])

,CHAS,RM,DIS,B,LSTAT,target
0,0.0,6.575,0.542096,1.000000,-1.275260,24.0
1,0.0,6.421,0.623954,1.000000,-0.263711,21.6
2,0.0,7.185,0.623954,0.989737,-1.627858,34.7
3,0.0,6.998,0.707895,0.994276,-2.153192,33.4
4,0.0,7.147,0.707895,1.000000,-1.162114,36.2
5,0.0,6.430,0.707895,0.992990,-1.200048,28.7
6,0.0,6.012,0.671500,0.996722,0.248456,22.9
7,0.0,6.172,0.700059,1.000000,0.968416,27.1
8,0.0,5.631,0.709276,0.974104,1.712312,16.5
9,0.0,6.004,0.743201,0.974305,0.779802,18.9


## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

Next, calculate the mean of the MSE over the 5 cross-validations and compare and contrast with the result from the train-test-split case.

##  Summary 

Congratulations! You now practiced your knowledge on k-fold crossvalidation!